# Exploratory Data Analysis (EDA) of the NOAA Severe Weather Data Inventory

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

In [3]:
# Commands to setup the notebook 
%load_ext autoreload
%autoreload 2

# Maximum number of lines to display
pd.options
pd.options.display.max_rows = 50

In [4]:
folder_path = 'storm_data'

# Load the data
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Read all the dfs into a list
dfs = [df for df in (pd.read_csv(os.path.join(folder_path, file), low_memory=False) for file in tqdm(csv_files)) if not df.empty]
print(f"There are {len(dfs)} non-empty dataframes in the list")

100%|██████████| 203/203 [00:08<00:00, 23.63it/s]

There are 180 non-empty dataframes in the list


In [5]:
columns_list = {tuple(df.columns) for df in dfs}
print(f"As expected there are {len(columns_list)} sets of columns since there are the following different dataset types; details, locations and fatalities")

As expected there are 3 sets of columns since there are the following different dataset types; details, locations and fatalities


In [ ]:
# Combine the dfs together by using 'episode_id' and 'event_id' as the key (as described in the pdf)
combined_df = dfs[0]

# Loop through the rest of the dfs and merge them with the combined_df, Event fatalities doesn't have episode_id
for df in dfs[1:]:
    if 'episode_id' in df.columns and 'episode_id' in combined_df.columns:
        join_cols = ['episode_id', 'event_id']
    elif 'event_id' in df.columns and 'event_id' in combined_df.columns:
        join_cols = ['event_id']
    else:
        # No matching keys, skip
        continue
    combined_df = pd.merge(combined_df, df, on=join_cols, how='outer')

print("Final shape:", combined_df.shape)

Final shape: (22, 11)


In [10]:
# Concatenate the dfs together
concat_df = pd.concat(dfs, axis=0)
print("Final shape:", concat_df.shape)

Final shape: (3690781, 70)


In [27]:
concat_df.head()

,FAT_YEARMONTH,FAT_DAY,FAT_TIME,FATALITY_ID,EVENT_ID,FATALITY_TYPE,FATALITY_DATE,FATALITY_AGE,FATALITY_SEX,FATALITY_LOCATION,...,DATA_SOURCE,YEARMONTH,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,LON2
0,195503.0,16.0,1430.0,1005348.0,10075750,D,03/16/1955 14:30:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,195503.0,1.0,152.0,1005349.0,10083328,D,03/01/1955 01:52:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,195503.0,13.0,2100.0,1005350.0,10117464,D,03/13/1955 21:00:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,195503.0,22.0,1430.0,1005351.0,10117756,D,03/22/1955 14:30:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,195504.0,21.0,141.0,1005352.0,9984179,D,04/21/1955 01:41:00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
concat_df.columns

Index(['FAT_YEARMONTH', 'FAT_DAY', 'FAT_TIME', 'FATALITY_ID', 'EVENT_ID',
       'FATALITY_TYPE', 'FATALITY_DATE', 'FATALITY_AGE', 'FATALITY_SEX',
       'FATALITY_LOCATION', 'EVENT_YEARMONTH', 'BEGIN_YEARMONTH', 'BEGIN_DAY',
       'BEGIN_TIME', 'END_YEARMONTH', 'END_DAY', 'END_TIME', 'EPISODE_ID',
       'STATE', 'STATE_FIPS', 'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE',
       'CZ_FIPS', 'CZ_NAME', 'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE',
       'END_DATE_TIME', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
       'SOURCE', 'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY',
       'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO',
       'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME',
       'BEGIN_RANGE', 'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE',
       'END_AZIMUTH', 'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT',
       'END_LON', 'EPISODE_NARRATIVE', 'EVENT_NARRATI

In [33]:
assert 'EVENT_ID' in concat_df.columns and 'EPISODE_ID' in concat_df.columns, "The columns 'EVENT_ID' and 'EPISODE_ID' are not in the dataframe"

In [23]:
# check out a random dataframe
dfs[50].head(5)

,FAT_YEARMONTH,FAT_DAY,FAT_TIME,FATALITY_ID,EVENT_ID,FATALITY_TYPE,FATALITY_DATE,FATALITY_AGE,FATALITY_SEX,FATALITY_LOCATION,EVENT_YEARMONTH
0,195807,1,1510,1005456,10055208,D,07/01/1958 15:10:00,NaN,NaN,NaN,195807
1,195808,4,1930,1005457,10055429,D,08/04/1958 19:30:00,NaN,NaN,NaN,195808
2,195809,7,1600,1005458,10040343,D,09/07/1958 16:00:00,NaN,NaN,NaN,195809
3,195810,19,735,1005459,9984944,D,10/19/1958 07:35:00,NaN,NaN,NaN,195810
4,195810,10,1905,1005460,10048193,D,10/10/1958 19:05:00,NaN,NaN,NaN,195810
